In [3]:
import pandas as pd
from tqdm import tqdm 
import numpy as np
import urllib
from zipfile import ZipFile
import psycopg2
main_path = '/home/fs01/spec1142/Emma/GateKeepers/download_PatentsView/'

f = open('/home/fs01/spec1142/Emma/GateKeepers/' + "database.txt", "r")
user , password = f.read().split()


## Download PatentsView tables

In [3]:
## all PatentsView tables are available at this link: 
## https://patentsview.org/download/data-download-tables

patents_path = "https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip"
inventors_path = "https://s3.amazonaws.com/data.patentsview.org/download/g_inventor_disambiguated.tsv.zip"
locations_path = "https://s3.amazonaws.com/data.patentsview.org/download/g_location_disambiguated.tsv.zip"
assignees_path = 'https://s3.amazonaws.com/data.patentsview.org/download/g_assignee_disambiguated.tsv.zip'
US_citations_path = 'https://s3.amazonaws.com/data.patentsview.org/download/g_us_patent_citation.tsv.zip'
wipo_path = 'https://s3.amazonaws.com/data.patentsview.org/download/g_wipo_technology.tsv.zip'
cpc_title = 'https://s3.amazonaws.com/data.patentsview.org/download/g_cpc_title.tsv.zip'
patent_cpcs = 'https://s3.amazonaws.com/data.patentsview.org/download/g_cpc_current.tsv.zip'
non_patent_citations = "https://zenodo.org/record/8169364/files/_pcs_oa.csv?download=1"
patent_applications = "https://s3.amazonaws.com/data.patentsview.org/download/g_application.tsv.zip"



In [4]:
##download data from the url and unzip file 

%%time

def download_data(url):

    """
    This function downloads a zip file from a specified URL and extracts its contents to a specified directory.

    Parameters:
    url (str): The URL of the zip file to be downloaded.

    Note:
    - The function save the zip file to the directory specified by `main_path + patents_path.zip`.
    """
    
    urllib.request.urlretrieve(url, main_path + 'patents_path.zip') # any dir to save
    with ZipFile(main_path +  'patents_path.zip' , 'r') as zObject:
        zObject.extractall(path=main_path)


url = patent_applications
download_data(url)

CPU times: user 110 ms, sys: 147 ms, total: 256 ms
Wall time: 5.12 s


('/home/fs01/spec1142/Emma/Download_PatentsView/patents_path.zip',
 <http.client.HTTPMessage at 0x7f66604c9df0>)

## Load data in the database 

### Create table

In [35]:
## create table 
table_name = "documentid_uspto"

## define columns of the table
schema = """CREATE TABLE  documentid_uspto ( 
   rf_id VARCHAR(15) ,
   lang VARCHAR(3),
   appno_doc_num VARCHAR(15),
   appno_date DATE,
   pgpub_doc_num VARCHAR(15),
   pgpub_date DATE,
   grant_doc_num VARCHAR(15),
   grant_date DATE,
   grant_country VARCHAR(3) );"""


def create_table(table_name, schema, drop_if_exists):

    """
    This function creates a table in a PostgreSQL database with the specified schema, and drops the table if it already exists.

    Parameters:
    table_name (str): The name of the table to be created in the PostgreSQL database.
    schema (str): The SQL schema defining the structure of the table.
    drop_if_exists (bool): A flag indicating whether to drop the table if it already exists before creating it.

    Returns:
    str: A success message if the table is created successfully.

    Note:
    - The function assumes that the `user` and `password` variables are defined elsewhere in the code.
    - The function commits the transaction and closes the database connection after creating the table.
    """

    conn = psycopg2.connect("user=" + user + " password=" + password)
    cursor = conn.cursor()

    if drop_if_exists == True:
        cursor.execute("DROP TABLE IF EXISTS " + table_name)
    
    #Creating table as per requirement
    sql = schema
    
    cursor.execute(sql)
    conn.commit()

    #Closing the connection
    conn.close()

    return "Table created successfully........"



Table created successfully........


### Load data

In [36]:
## load data into the table 

## input path to the data and table name 
data_path = main_path + "/documentid.tsv"
table_name = "documentid_uspto"


def load_data_into_table(table_name, data_path):

    """
    This function loads data from a TSV file into a specified table in a PostgreSQL database.

    Parameters:
    table_name (str): The name of the table in the PostgreSQL database to load data into.
    data_path (str): The file path of the TSV file containing the data to be loaded.

    Returns:
    str: A success message if the data is loaded successfully.

    Note:
    - The function assumes that the `user` and `password` variables are defined elsewhere in the code.
    - The function commits the transaction and closes the database connection after loading the data.
    """

    ## load data into the table
    conn = psycopg2.connect("user=" + user + " password=" + password)
    cursor = conn.cursor()
    
    with open(data_path) as f:
        cursor.copy_expert("COPY " + table_name + " FROM STDIN WITH DELIMITER E'\t' CSV HEADER", f)
    
    conn.commit()
    conn.close()

    return "Data loaded successfully.........."
                

### Index table

In [ ]:
## chose the columns to index 
table_name = 'inventors_PatentsView'
index_columns = ['inventor_id']

def index_table(table_name , index_columns):

    """
    This function creates an index on each specified column in a table in a PostgreSQL database.

    Parameters:
    table_name (str): The name of the table to be indexed in the PostgreSQL database.
    index_columns (list): A list of column names to create indexes on.

    Returns:
    str: A success message if the table is indexed successfully.

    Note:
    - The function assumes that the `user` and `password` variables are defined elsewhere in the code.
    - The function commits the transaction and closes the database connection after creating the indexes.
    """
    
    #establishing the connection
    conn = psycopg2.connect("user=" + user + " password=" + password)
    
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    
    ## index each column
    for index_column in index_columns:
    
        #Index table as per requirement
        sql ='''CREATE INDEX ''' + index_column + '_' + table_name + ''' ON '''+ table_name +'''(''' + index_column + ''');'''
        cursor.execute(sql)
        
    conn.commit()
    #Closing the connection
    conn.close()
    
    return "Table indexed successfully........"

## Query

In [6]:
## load data into the table
conn = psycopg2.connect("user=" + user + " password=" + password)
cursor = conn.cursor()
    
text = """SELECT * FROM inventors_PatentsView WHERE patent_id = '10167450';"""
    
cursor.execute(text)
res = cursor.fetchall()

cursor.close()

res

[('10167450',
  0,
  'fl:pa_ln:czernichow-1',
  'Paul',
  'Czernichow',
  'M',
  '173808ad-16c8-11ed-9b5f-1234bde3cd05',
  'Czernichow'),
 ('10167450',
  2,
  'fl:ph_ln:ravassard-1',
  'Philippe',
  'Ravassard',
  'M',
  '173808ad-16c8-11ed-9b5f-1234bde3cd05',
  'Ravassard'),
 ('10167450',
  1,
  'fl:ra_ln:scharfmann-1',
  'Raphael',
  'Scharfmann',
  'M',
  '173808ad-16c8-11ed-9b5f-1234bde3cd05',
  'Scharfmann')]

## Download Patstat assignee classification from Google Cloud

In [12]:
## download files from Google Cloud 

from google.cloud import storage
import subprocess
import pandas as pd

path_to_private_key = '/home/fs01/spec1142/Emma/' + "openalex-lee-c532eb059285.json"
client_storage = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)

bucket_name = 'openalex-lee'
prefix = 'Patstat/'
dl_dir = main_path + "Download_PatentsView/assignees_Patstat/" 
       
bucket = client_storage.get_bucket(bucket_or_name=bucket_name)
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs:
    filename = blob.name.replace('/', '_') 
    blob.download_to_filename(dl_dir + filename)  # Download

In [13]:
## merge files together

data_path = '/home/fs01/spec1142/Emma/Download_PatentsView/'

cmd = "ls " + data_path + "assignees_Patstat/"
files_to_upload = subprocess.check_output(cmd,  shell=True)
files_to_upload = files_to_upload.decode().split("\n")
files_to_upload.remove('')

table = pd.concat( [ pd.read_csv(data_path + "assignees_Patstat/" + file , delimiter = "\t" ) for file in files_to_upload ] )

In [15]:
table

,person_id,appln_id
0,5229210,306945239
1,15331669,23972965
2,12494727,47315474
3,9348991,273068679
4,13502439,23968941
...,...,...
2983788,80986870,545400751
2983789,79276293,544655559
2983790,78433347,545124265
2983791,58296180,457588798


In [16]:
## save files

table.to_csv(data_path + "person_patents_Patstat.tsv" , sep = "\t" , index = False)